# Problem Set 3 


In this problem set, you'll implement a version of the 6.0001 word game!

## Notes

- 80% Complete. Went through problem 4 and stopped at problem 5. 
- Did not incoporate any of the pre-existing testing schemes, but did build and pass my own unit tests throughout development.

## Dealing

* player is deatlh hand of `HAND_SIZE` letters of alphabet, @random. Possible to receive multiple instances of same letter (draw w replacement).

* Player arranges hand into as many words they want, but can only use each letter at most once.

* Some letters remain unused, 

## Scoring

* The score for hand is sum of score for each word formed.

* Score for word is **product** of:
    * the sum of points for letters in the word
    * maximum( [7\*word_length -3\*(n - word_length)], 1).
        * word_lenght = number letters used in word
        * n = number of letters avail in current hand
        
* Letters scored like scrabble: A = 1, B = 3, C = 3, D = 2, E = 1 and so on. there is a pre-defined map `SCRABBLE_LETTER_VALUES` that defines how much each letter is worth.

* Example:
    * n = 6 and hand includes 1 'w', 2 'e's, 1 'd' (and to other letters), playing the word 'weed' would be worth:
        * sum of points for letters in weed: (4 + 1 + 1 + 2) = 8
        * word_length = 4
        * n = total letters = 6:
        * max(7*4 - 3*(6-4), 1) = max(22,1) = 22
        * 22*8 = 176.

# Importing necessary code

File ps3.py includes code to implement this solution. We are going to piece-by-piece import it into this notebook.

The **test_ps3.py** code is a separate file we can use to test our code along the way. It was provided by the homework makers.

In [1]:
# 6.0001 Problem Set 3
#
# The 6.0001 Word Game
# Created by: Kevin Luu <luuk> and Jenna Wiens <jwiens>
#
# Name          : Jaime Lopez-Merizalde
# Collaborators : <your collaborators>
# Time spent    : <total time>

# imports
import math
import random
import string

# constants
VOWELS = 'aeiou'
CONSONANTS = 'bcdfghjklmnpqrstvwxyz'
HAND_SIZE = 7

SCRABBLE_LETTER_VALUES = {
    'a': 1, 'b': 3, 'c': 3, 'd': 2, 'e': 1, 'f': 4, 'g': 2, 'h': 4, 'i': 1, 'j': 8, 'k': 5, 'l': 1, 'm': 3, 'n': 1, 'o': 1, 'p': 3, 'q': 10, 'r': 1, 's': 1, 't': 1, 'u': 1, 'v': 4, 'w': 4, 'x': 8, 'y': 4, 'z': 10
}

WORDLIST_FILENAME = "words.txt"

## Some Helper Code

In [63]:
def load_words():
    """
    Returns a list of valid words. Words are strings of lowercase letters.
    
    Depending on the size of the word list, this function may
    take a while to finish.
    """
    
    print("Loading word list from file...")
    # inFile: file
    inFile = open(WORDLIST_FILENAME, 'r')
    # wordlist: list of strings
    wordlist = []
    for line in inFile:
        wordlist.append(line.strip().lower())
    print("  ", len(wordlist), "words loaded.")
    return wordlist

In [64]:
# unit test load_words
returned = len(load_words())
expected = 83667
print("words returned: ",returned)
print("words expected: ", expected)

if (returned == expected):
    print("PASSED")
else: 
    print("FAILED")

Loading word list from file...
   83667 words loaded.
words returned:  83667
words expected:  83667
PASSED


In [4]:
def get_frequency_dict(sequence):
    """
    Returns a dictionary where the keys are elements of the sequence
    and the values are integer counts, for the number of times that
    an element is repeated in the sequence.

    sequence: string or list
    return: dictionary
    """
    
    # freqs: dictionary (element_type -> int)
    freq = {}
    for x in sequence:
        freq[x] = freq.get(x,0) + 1
    return freq
	

In [54]:
# unit test get_frequency_dict

def unit_test(expected_dict, returned_dict):
    
    print("returned_dict", returned_dict)
    print("expected_dict", expected_dict)
    
    pass_flag = False
    for key in expected_dict:
        if (key in returned_dict and expected_dict[key]==returned_dict[key]):
            pass_flag = True
        else: 
            print("FAILED")
            break
        
    if pass_flag:
        print("PASSED")
        
test_string = "statistics"
expected_dict = {'s': 3, 't': 3, 'a': 1, 'i': 2, 'c': 1}
returned_dict = get_frequency_dict(test_string)
unit_test(expected_dict, returned_dict)
        
test_string = ['s','t','a','t','i','s','t','i','c','s']
expected_dict = {'s': 3, 't': 3, 'a': 1, 'i': 2, 'c': 1}
returned_dict = get_frequency_dict(test_string)
unit_test(expected_dict, returned_dict)

returned_dict {'s': 3, 't': 3, 'a': 1, 'i': 2, 'c': 1}
expected_dict {'s': 3, 't': 3, 'a': 1, 'i': 2, 'c': 1}
PASSED
returned_dict {'s': 3, 't': 3, 'a': 1, 'i': 2, 'c': 1}
expected_dict {'s': 3, 't': 3, 'a': 1, 'i': 2, 'c': 1}
PASSED


## Problem 1: Scoring a word

In [5]:
def get_word_score(word, n):
    """
    Returns the score for a word. Assumes the word is a
    valid word.

    You may assume that the input word is always either a string of letters, 
    or the empty string "". You may not assume that the string will only contain 
    lowercase letters, so you will have to handle uppercase and mixed case strings 
    appropriately. 

	The score for a word is the product of two components:

	The first component is the sum of the points for letters in the word.
	The second component is the larger of:
            1, or
            7*wordlen - 3*(n-wordlen), where wordlen is the length of the word
            and n is the hand length when the word was played

	Letters are scored as in Scrabble; A is worth 1, B is
	worth 3, C is worth 3, D is worth 2, E is worth 1, and so on.

    word: string
    n: int >= 0
    returns: int >= 0
    """
    
    #==========CODE===========
    
    # string to lower case
    word=word.lower()
    
    # factor 1: sum of points for letters in word
    letter_freq=get_frequency_dict(word) 
    
    # map to values:
    dict_freq.keys()

    factor_1 = sum([SCRABBLE_LETTER_VALUES[k]*letter_freq[k] for k in letter_freq])
    
    # factor 2: max( 1, 7*wordlen - 3(n - wordlen))
    wordlen=len(word)
    factor_2=max(1, 7 * wordlen - 3 * (n - wordlen))
    
    score=factor_1 * factor_2
    
    return score
    
    #==========CODE===========

In [56]:
def unit_test(expected, returned):
    
    print("returned", returned)
    print("expected", expected)
    
    pass_flag = False
    
    if (expected==returned):
        pass_flag = True
    else: 
        print("FAILED")
        
    if pass_flag:
        print("PASSED")
        
#====pre-work=====
dict_freq={'a':1, 'p':2, 'l':1, 'e':1}
dict_value={'a':1, 'p': 3, 'l':1, 'e':1}

factor_1=sum([dict_value[k]*dict_freq[k] for k in dict_freq])
wordlen=5
n=10 #cards
factor_2=max(1, 7 * wordlen - 3 * (n - wordlen))
#max(1,7*5-3*(10-5)) = max(1, 35-15) = max(1,20) = 20
score=factor_1 * factor_2
print('score: ', score)
#====pre-work=====

# words = {("", 7):0, ("it", 7):2, ("was", 7):54, ("weed", 6):176,
#              ("scored", 7):351, ("WaYbILl", 7):735, ("Outgnaw", 7):539,
#              ("fork", 7):209, ("FORK", 4):308}
                    
test = "it"
n = 7
expected = 2
returned = get_word_score(test,n)
unit_test(expected, returned)

score:  180
returned 2
expected 2
PASSED


In [8]:
#
# Make sure you understand how this function works and what it does!
#
def display_hand(hand):
    """
    Displays the letters currently in the hand.

    For example:
       display_hand({'a':1, 'x':2, 'l':3, 'e':1})
    Should print out something like:
       a x x l l l e
    The order of the letters is unimportant.

    hand: dictionary (string -> int)
    """
    
    # JL: for each letter in the keys of the hand (which is a freq map of the letters avail)
    for letter in hand.keys():
        # JL: take j = 0 to freq_of_letter-1 
        for j in range(hand[letter]):
            # JL: print that letter, then add a space before printing again
             print(letter, end=' ')      # print all on the same line
    print()                              # print an empty line


### Unit test: display_hand

You can put a unit test here for the function above. I'm going to skip that part.

In [9]:
#
# Make sure you understand how this function works and what it does!
# You will need to modify this for Problem #4.
#
def deal_hand(n):
    """
    Returns a random hand containing n lowercase letters.
    ceil(n/3) letters in the hand should be VOWELS (note,
    ceil(n/3) means the smallest integer not less than n/3).

    Hands are represented as dictionaries. The keys are
    letters and the values are the number of times the
    particular letter is repeated in that hand.

    n: int >= 0
    returns: dictionary (string -> int)
    """
    # JL: start an empty dict for the hand to be filled in later by random.
    # JL: this func determines what you are DEALT at the beg of the game!
    hand={}
    # JL: one requirement is how many vowels there should be. It's the ceil of 1/3 
    # letters passed out in any hand.
    num_vowels = int(math.ceil(n / 3))

    # for each vowel-draw
    for i in range(num_vowels):
        # select a random one from the pre-pop list of vowels above
        x = random.choice(VOWELS)
        # set the dictionary key-val pair: x->what value is 
        # there (if nothing return 0, that's what hand.get(x,0) does) and +1 to it.
        hand[x] = hand.get(x, 0) + 1
    
    # for each value left in the hand (n - num_vowels)
    for i in range(num_vowels, n):   
        # select a random one from the pre-pop list of consonants above 
        x = random.choice(CONSONANTS)
        # set the dictionary key-val pair: x->what value is 
        # there (if nothing return 0, that's what hand.get(x,0) does) and +1 to it. 
        hand[x] = hand.get(x, 0) + 1
    
    return hand

### Unit test: deal_hand(n)

In [10]:
def unit_test(expected, returned):
    
    print("returned", returned)
    print("expected", expected)
    
    pass_flag = False
    
    if (expected==returned):
        pass_flag = True
    else: 
        print("FAILED")
        
    if pass_flag:
        print("PASSED")
        
#====pre-work=====
#====pre-work=====

# note: this unit test requires only printing that the deal hand contains only nearly a 1/3 of values as RND vwls
# and 2/3 as RND consonants
test = 10
expected = [int(math.ceil(test/3)), int(math.floor(2*test/3))]
returned = deal_hand(test)
unit_test(expected, returned)

returned {'a': 1, 'u': 2, 'e': 1, 'p': 1, 'j': 3, 'l': 1, 't': 1}
expected [4, 6]
FAILED


## Problem 2: Update hand by removing letters

In [47]:
def update_hand(hand, word):
    """
    Does NOT assume that hand contains every letter in word at least as
    many times as the letter appears in word. Letters in word that don't
    appear in hand should be ignored. Letters that appear in word more times
    than in hand should never result in a negative count; instead, set the
    count in the returned hand to 0 (or remove the letter from the
    dictionary, depending on how your code is structured). 

    Updates the hand: uses up the letters in the given word
    and returns the new hand, without those letters in it.

    Has no side effects: does not modify hand.

    word: string
    hand: dictionary (string -> int)    
    returns: dictionary (string -> int)
    """

    #==========CODE===========
    
    # get the frequency of the letters in word
    word_freq=get_frequency_dict(word)
    
    # begin walking through word_freq and removing 
    # the detailed quantity from 'hand' 
    # a copy has to be made, because dictionaries have a global scope
    # anything passed down and chnaged here is 
    hand_copy=hand.copy()
    for char in word_freq.keys():
        hand_copy[char]=max(0, hand_copy.get(char, 0) - word_freq[char])
        
    
    # note: this should be max(0, hand@character - word_freq@character)
    # if the latter is negative, representing that there are too many letters, 
    # it simply takes them all away as a punishment, 
    # note: if the letter is NOT in hand, then do nothing! 
    
    return hand_copy
    #==========CODE===========

### Unit Test: update_hand(hand, word)

Follow the testing example given in the pdf.

In [11]:
def unit_test(expected, returned):
    
    print("returned", returned)
    print("expected", expected)
    
    pass_flag = False
    
    if (expected==returned):
        pass_flag = True
    else: 
        print("FAILED")
        
    if pass_flag:
        print("PASSED")
        
#====pre-work=====
hand={'a':1, 'q':1, 'l':2, 'm':1, 'u':1, 'i':1}
display_hand(hand)

new_hand=update_hand(hand, 'quail')
display_hand(new_hand)

display_hand(hand)
print()
#====pre-work=====

expected = {'a':0, 'q':0, 'l':1, 'm':1, 'u':0, 'i':0}
returned = new_hand
unit_test(expected, returned)

a q l l m u i 
l m 
a q l l m u i 

returned {'a': 0, 'q': 0, 'l': 1, 'm': 1, 'u': 0, 'i': 0}
expected {'a': 0, 'q': 0, 'l': 1, 'm': 1, 'u': 0, 'i': 0}
PASSED


## Problem 3: Test word validity

In [38]:
#
# Problem #3: Test word validity
#
def is_valid_word(word, hand, word_list):
    """
    Returns True if word is in the word_list and is entirely
    composed of letters in the hand. Otherwise, returns False.
    Does not mutate hand or word_list.
   
    word: string
    hand: dictionary (string -> int)
    word_list: list of lowercase strings
    returns: boolean
    """
    
    #==========CODE===========
    
    # current status
    is_valid_word_ = False
    
    # to lower case
    word=word.lower()
    
    # get frequency dict of the word
    word_freq = get_frequency_dict(word)
    
    # check if in word_list
    if (not word in word_list):
        return False
    
    else :
        # loop through letters in word and decide if there are enough
        # in the hand to support it. 
        # return false as soon as we can't support this word.
        for k in word_freq.keys():
            if (word_freq[k] > hand[k]):
                return False
        
        
    #==========CODE===========
    # code reaches this return statement if it passes the previous checks. 
    return True

### Unit Test: is_valid_word(word, hand, word_list)

 

In [13]:
def unit_test(expected, returned):
    
    print("expected", expected)
    print("returned", returned)
      
    pass_flag = False
    
    if (expected==returned):
        pass_flag = True
    else: 
        print("FAILED")
        
    if pass_flag:
        print("PASSED")
        
#====pre-work=====
word = "bird"
wordlist = load_words()
hand = {'b':1, 'i':1, 'r':1, 'd':1}
#====pre-work=====

expected = True
returned = is_valid_word(word, hand, wordlist)
unit_test(expected, returned)

Loading word list from file...
   83667 words loaded.
expected True
returned True
PASSED


### Unit Test: is_valide_word(...)

Could make more unit test to see when words are not in the dictionary, but for the sake of time, I'm passing over this.

In [14]:
wordlist = load_words()

Loading word list from file...
   83667 words loaded.


## Problem 4: Playing a hand

In [55]:
#
# Problem #4: Playing a hand
#
def calculate_handlen(hand):
    """ 
    Returns the length (number of letters) in the current hand.
    
    hand: dictionary (string-> int)
    returns: integer
    """
    
    #====Code=====
    handlength=0
    
    for k in hand.keys() :
        handlength+=hand[k]

    return handlength
    #====Code=====

## Unit Test: calculate_handlen(hand):

In [58]:
def unit_test(expected, returned):
    
    print("expected", expected)
    print("returned", returned)
      
    pass_flag = False
    
    if (expected==returned):
        pass_flag = True
    else: 
        print("FAILED")
        
    if pass_flag:
        print("PASSED")
        
#====pre-work=====
# word = "bird"
# wordlist = load_words()
hand = {'b':1, 'i':1, 'r':1, 'd':1}
hand = {'b':0, 'i':0, 'r':0, 'd':0}
#====pre-work=====

expected = 4
returned = calculate_handlen(hand)
unit_test(expected, returned)

expected 4
returned 0
FAILED


In [61]:
def play_hand(hand, word_list):
    """
    Allows the user to play the given hand, as follows:

    * The hand is displayed.
    
    * The user may input a word.

    * When any word is entered (valid or invalid), it uses up letters
      from the hand.

    * An invalid word is rejected, and a message is displayed asking
      the user to choose another word.

    * After every valid word: the score for that word is displayed,
      the remaining letters in the hand are displayed, and the user
      is asked to input another word.

    * The sum of the word scores is displayed when the hand finishes.

    * The hand finishes when there are no more unused letters.
      The user can also finish playing the hand by inputing two 
      exclamation points (the string '!!') instead of a word.

      hand: dictionary (string -> int)
      word_list: list of lowercase strings
      returns: the total score for the hand
      
    """
    
    # BEGIN PSEUDOCODE <-- Remove this comment when you implement this function
    # Keep track of the total score
    total_score = 0
    
    # As long as there are still letters left in the hand:
    while calculate_handlen(hand)>0:
        
        # Display the hand
        print()
        print("Current Hand")
        display_hand(hand)
        
        # Ask user for input
        word = input("Enter a word for your letters: ")
        
        # If the input is two exclamation points:
        if (word=="!!"):
            break
            # End the game (break out of the loop)

            
        # Otherwise (the input is not two exclamation points):
        else:
            # If the word is valid:
            if (is_valid_word(word, hand, word_list)):
                
                # Tell the user how many points the word earned,
                # and the updated total score
                points_earned=get_word_score(word, HAND_SIZE)
                total_score+=points_earned
                print("Your word earned: ", points_earned)
                print("The total score is now: ", total_score)
                    
            # Otherwise (the word is not valid):
            else :
                print("Invalid word! Do something else.")
                # Reject invalid word (print a message)
                
            # update the user's hand by removing the letters of their inputted word
            hand = update_hand(hand, word)

    # Game is over (user entered '!!' or ran out of letters),
    # so tell user the total score
    print("Game Over. Your total score was: ", total_score)

    # Return the total score as result of function
    return total_score


## Unit Test: play_hand(hand, word_list):

In [1]:
def unit_test(expected, returned):
    
    print("expected", expected)
    print("returned", returned)
      
    pass_flag = False
    
    if (expected==returned):
        pass_flag = True
    else: 
        print("FAILED")
        
    if pass_flag:
        print("PASSED")

#====pre-work=====
# word = "bird"
wordlist = load_words()
hand = {'b':1, 'i':2, 'r':2, 'd':2, 't':1}
#====pre-work=====
expected=228 # this is not verified independently 
returned=play_hand(hand, wordlist)
unit_test(expected, returned) 

NameError: name 'load_words' is not defined

## Problem 5: Playing a game

In [11]:
#
# procedure you will use to substitute a letter in a hand
#

def substitute_hand(hand, letter):
    """ 
    Allow the user to replace all copies of one letter in the hand (chosen by user)
    with a new letter chosen from the VOWELS and CONSONANTS at random. The new letter
    should be different from user's choice, and should not be any of the letters
    already in the hand.

    If user provide a letter not in the hand, the hand should be the same.

    Has no side effects: does not mutate hand.

    For example:
        substitute_hand({'h':1, 'e':1, 'l':2, 'o':1}, 'l')
    might return:
        {'h':1, 'e':1, 'o':1, 'x':2} -> if the new letter is 'x'
    The new letter should not be 'h', 'e', 'l', or 'o' since those letters were
    already in the hand.
    
    hand: dictionary (string -> int)
    letter: string
    returns: dictionary (string -> int)
    """
    
    pass  # TO DO... Remove this line when you implement this function


In [12]:
def play_game(word_list):
    """
    Allow the user to play a series of hands

    * Asks the user to input a total number of hands

    * Accumulates the score for each hand into a total score for the 
      entire series
 
    * For each hand, before playing, ask the user if they want to substitute
      one letter for another. If the user inputs 'yes', prompt them for their
      desired letter. This can only be done once during the game. Once the
      substitue option is used, the user should not be asked if they want to
      substitute letters in the future.

    * For each hand, ask the user if they would like to replay the hand.
      If the user inputs 'yes', they will replay the hand and keep 
      the better of the two scores for that hand.  This can only be done once 
      during the game. Once the replay option is used, the user should not
      be asked if they want to replay future hands. Replaying the hand does
      not count as one of the total number of hands the user initially
      wanted to play.

            * Note: if you replay a hand, you do not get the option to substitute
                    a letter - you must play whatever hand you just had.
      
    * Returns the total score for the series of hands

    word_list: list of lowercase strings
    """
    
    print("play_game not implemented.") # TO DO... Remove this line when you implement this function
    


In [14]:
#
# Build data structures used for entire session and play game
# Do not remove the "if __name__ == '__main__':" line - this code is executed
# when the program is run directly, instead of through an import statement
#
if __name__ == '__main__':
    word_list = load_words()
    play_game(word_list)

Loading word list from file...
   83667 words loaded.
play_game not implemented.


## Test code
This import willl call tye **test_ps3.py** file which includes many pre-written tests for the work. As we progress, this file will pass tests until we can play games successfully. It is useful for making sure we are progressing during the development process.

In [185]:
# Comment out this code so it does not run along with your work.
import test_ps3
exec(open('test_ps3.py').read())

Loading word list from file...
   83667 words loaded.
----------------------------------------------------------------------
Testing get_word_score...
FAILURE: test_get_word_score()
	Expected 0 points but got 'None' for word '', n=7
FAILURE: test_get_word_score()
	Expected 2 points but got 'None' for word 'it', n=7
FAILURE: test_get_word_score()
	Expected 54 points but got 'None' for word 'was', n=7
FAILURE: test_get_word_score()
	Expected 176 points but got 'None' for word 'weed', n=6
FAILURE: test_get_word_score()
	Expected 351 points but got 'None' for word 'scored', n=7
FAILURE: test_get_word_score()
	Expected 735 points but got 'None' for word 'WaYbILl', n=7
FAILURE: test_get_word_score()
	Expected 539 points but got 'None' for word 'Outgnaw', n=7
FAILURE: test_get_word_score()
	Expected 209 points but got 'None' for word 'fork', n=7
FAILURE: test_get_word_score()
	Expected 308 points but got 'None' for word 'FORK', n=4
-------------------------------------------------------------